<a href="https://colab.research.google.com/github/whoami-Lory271/thesis-project/blob/TiDE_SimTS/TiDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TiDE

In [1]:
!pip install ipdb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00


In [13]:
import torch
import torch.nn as nn
import ipdb


args = {
    "dataset_name" : "ETTm1",
    "path_data" : "",
    "hidden_size" : 128,
    "temporal_features" : 4,
    "temporal_width" : 4,
    "n_encoder_layers" : 2,
    "n_decoder_layers" : 2,
    "decoder_output_size" : 32,
    "decoder_temporal_hidden_size" : 128,
    "dropout" : 0.1,
    # Train arguments
    "batch_size" : 4
}


class ResidualBlock(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.0):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Dropout(dropout)
        )
        self.residual = nn.Linear(input_size, output_size)
        self.layer_norm = nn.LayerNorm(output_size)

    def forward(self, x):
        output = self.block(x) + self.residual(x)
        output = self.layer_norm(output)
        return output


class TemporalResidualBlock(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TemporalResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        self.residual = nn.Linear(input_size, 1)

    def forward(self, x):
        output = self.block(x) + self.residual(x)
        return output


class FeatureProjectionBlock(nn.Module):
    def __init__(self, input_size, hidden_size, temporal_width=4):
        super(FeatureProjectionBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, temporal_width)
        )
        self.residual = nn.Linear(input_size, temporal_width)

    def forward(self, x):
        output = self.block(x) + self.residual(x)
        return output


class TiDE(nn.Module):
    def __init__(self, args, input_steps, output_steps):
        super(TiDE, self).__init__()
        self.input_size = input_steps
        self.output_size = output_steps
        self.hidden_size = args["hidden_size"]
        self.temporal_features = args["temporal_features"]
        self.temporal_width = args["temporal_width"]
        self.n_encoder_layers = args["n_encoder_layers"]
        self.n_decoder_layers = args["n_decoder_layers"]
        self.decoder_output_size = args["decoder_output_size"]
        self.decoder_temporal_hidden_size = args["decoder_temporal_hidden_size"]
        self.dropout = args["dropout"]

        encoder_layers = []
        for i in range(self.n_encoder_layers):
            if i == 0:
                residual_block = ResidualBlock(self.input_size+self.output_size*self.temporal_width, self.hidden_size, self.hidden_size, self.dropout)
            else:
                residual_block = ResidualBlock(self.hidden_size, self.hidden_size, self.hidden_size, self.dropout)
            encoder_layers.append(residual_block)
        self.dense_encoder = nn.Sequential(*encoder_layers)

        decoder_layers = []
        for i in range(self.n_decoder_layers):
            if i == self.n_decoder_layers - 1:
                residual_block = ResidualBlock(self.hidden_size, self.hidden_size, self.output_size*self.decoder_output_size, self.dropout)
            else:
                residual_block = ResidualBlock(self.hidden_size, self.hidden_size, self.hidden_size, self.dropout)
            decoder_layers.append(residual_block)
        self.dense_decoder = nn.Sequential(*decoder_layers)
        self.temporal_decoder = TemporalResidualBlock(self.decoder_output_size+self.temporal_width, self.decoder_temporal_hidden_size)

        self.feature_projection = FeatureProjectionBlock(self.temporal_features, self.hidden_size, self.temporal_width)

        self.residual = nn.Linear(self.output_size, self.output_size)

    def forward(self, x, feat):
        assert x.dim() == 3 and x.size(2) == 1
        ipdb.set_trace()
        feat_embedding = self.feature_projection(feat)
        feat_embedding = feat_embedding.reshape(feat_embedding.size(0), -1)
        input = torch.cat([x.squeeze(2), feat_embedding], dim=1)
        encoder_state = self.dense_encoder(input)
        decoder_outputs = self.dense_decoder(encoder_state)
        decoder_outputs = decoder_outputs.reshape(decoder_outputs.size(0), self.output_size, self.decoder_output_size)
        decoder_outputs = torch.cat([decoder_outputs, feat_embedding.reshape(feat_embedding.size(0), self.output_size, self.temporal_width)], dim=2)
        decoder_outputs = self.temporal_decoder(decoder_outputs)
        decoder_outputs = decoder_outputs + self.residual(x[:,:self.output_size].squeeze(2)).unsqueeze(2)
        return encoder_state, decoder_outputs


In [9]:
args

AttributeError: ignored

In [16]:
L = 10
H = 4
D = 3
R = 4
y = torch.rand(4, L, D)
x = torch.rand(4, L + H, R)
tide = TiDE(args, D, 2)
tide(y,x)

> <ipython-input-13-4a1d49a27cef>(113)forward()
    112         ipdb.set_trace()
--> 113         feat_embedding = self.feature_projection(feat)
    114         feat_embedding = feat_embedding.reshape(feat_embedding.size(0), -1)

ipdb> x.dim()
3
ipdb> x.size(1)
10
ipdb> x.size(2)
3



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/debugger.py", line 1075, in cmdloop
    sys.settrace(None)



--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


RuntimeError: ignored